# Guided Capstone Step 3

In [26]:
# sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# import json
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType
import decimal
import logging
# import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# First, let us place all of the data into one folder.

In [2]:
# Example S3 URI for later: 
# s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_CSV_Files/All_Ingested_CSV_Parquet/partition=T/part-00001-df714ede-7d68-427d-a5c2-cdbfd95cc993.c000.snappy.parquet

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Use the CSV Parser in Spark Transformation to Process the Source Data
spark = SparkSession.builder.master('local').appName('app').getOrCreate()  # Main entry point to Spark cluster

# # Create s3 resource object and use spark.conf.set() in order to work with S3 bucket.
# s3_resource = boto3.resource('s3')

spark.conf.set("fs.s3.endpoint", "s3-sa-east-1.amazonaws.com")

# Start Spark context manager --> used for creating RDD.
sc = spark.sparkContext  # Entry point to Spark application w/ help of Spark cluster.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
#  Populate trade dataset - first create output_path object
output_path = "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_CSV_Files/All_Ingested_CSV_Parquet/"

# Create parquet file object for trade data
trade_parquet_path = output_path + "partition=T"

# Read Trade Partition Dataset from temp location
trade_common = spark.read.parquet(trade_parquet_path)

# Select Necessary Columns for Trade Records (Remove and filter out unnecessary columns)
trade_df = trade_common.select("trade_dt", "symbol", "event_tm", "event_seq_nb",
                               "trade_pr",  "arrival_tm", "trade_pr")


# Apply Data Correction
def apply_latest_trade(df):
    """
    Inside exchange dataset, you can uniquely identify a record by the combination of trade_dt,
    symbol, exchange, event_tm, event_seq_nb.
    However, the exchange may correct an error in any submitted record by sending a new record w/
    the same unique ID.
    --> Such records will come w/ later "arrival_tm". Thus, you must ensure you only accept the one w/
    the most recent "arrival_tm".

    LOGIC:
     - If there are submitted records w/ same uniqueID, we will simply take the one w/ the
     most recent arrival_tm.
        ---> Operation involves aggregation to group dataset by unique ID s.t. a single function
        can be applied to a certain group entirely.
        ---> In Spark, we use groupBy operation to achieve this.

    NOTE:
    - I previously used ".show(1)", but it can either be ".take(1)" or what I currently have:
    ".limit(1).show()"
    - Alternative way is to do a ".groupBy(<Specify columns>).agg(func(max(<"arrival_tm">))".

    :param df: Spark Dataframe
    :return: After aggregating based on uniqueID, we will take the most recent "arrival_tm".
    """
    most_recent_trade = df.groupBy("trade_dt").agg(func.max("arrival_tm").alias("max_arrival_dtm"))
    return most_recent_trade


# Apply method
trade_corrected = apply_latest_trade(trade_df)

# Write Dataset Back To Parquet On AWS
trade_date = "2020-07-29"
trade_corrected.write.parquet(f"{output_path}trade/trade_dt={trade_date}")  # May have to manually create this?

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# NEXT:
# - Populate Quote dataset using same method.

# Quote parquet path
quote_parquet_path = output_path + "partition=Q"

# Read Trade Partition Dataset from temp location
quote_common = spark.read.parquet(quote_parquet_path)

# Select Necessary Columns for Quote Records
quote_df = quote_common.select("trade_dt", "symbol", "event_tm", "event_seq_nb",
                               "trade_pr",  "arrival_tm", "trade_pr")


# Apply Data Correction
def apply_latest_quote(df):
    """
    Inside exchange dataset, you can uniquely identify a record by the combination of quote_dt,
    symbol, exchange, event_tm, event_seq_nb.
    However, the exchange may correct an error in any submitted record by sending a new record w/
    the same unique ID.
    --> Such records will come w/ later "arrival_tm". Thus, you must ensure you only accept the one w/
    the most recent "arrival_tm".

    LOGIC:
     - If there are submitted records w/ same uniqueID, we will simply take the one w/ the
     most recent arrival_tm.
        ---> Operation involves aggregation to group dataset by unique ID s.t. a single function
        can be applied to a certain group entirely.
        ---> In Spark, we use groupBy operation to achieve this.

    NOTE:
    - I previously used ".show(1)", but it can either be ".take(1)" or what I currently have:
    ".limit(1).show()"

    :param df: Spark Dataframe
    :return: After aggregating based on uniqueID, we will take the most recent "arrival_tm".
    """
    # Aggregate by unique id (use groupBy) --> Obtain most recent "arrival_tm"
    most_recent_quote = df.groupBy("trade_dt").agg(func.max("arrival_tm").alias("max_arrival_dtm"))
    return most_recent_quote


# Apply method
quote_corrected = apply_latest_quote(quote_df)

# Write Dataset Back To Parquet On AWS
quote_date = "2020-07-29"
quote_corrected.write.parquet(f"{output_path}quote/quote_dt={quote_date}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Okay, we are done. Now, when we check S3 bucket, we should see are parquet files written as desired. :)